# ASSIGNMENT  2
## Team Number : 10

### Team Members :

   #### Manjunath Bhadrannavar : 01FB16ECS195
   #### Megha Kalal : 01FB16ECS203
   #### Nikhil V Revankar : 01FB16ECS230

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Spacy
import spacy
nlp = spacy.load('en_core_web_sm')

from nltk.corpus import wordnet as wn
import math

In [2]:
sents1 = ["I like fish fry with lunch whenever I am by the side of sea.",
          "He is visiting bank to deposit all his cash earned during the day It will earn interest overtime.",
          "Pure malt whiskey with a bit of soda can make you do what you are afraid of doing. It can make you brave for a while.",
          "It is a horse that is more faithful and powerful than my car. It loves me, cares for me, talks to me and makes me feel good when I am upset.",
          "I have a pen that writes like a dream. I carry it in all my examinations as the examiners need good hand-writing on the answer scripts.",
          "My driving license is my identity in USA as everything is linked to it."]

sents2 = ["Our professor  is ferocious i.e. if you make noise in classroom, he will fry you in front of everybody",
          "During flood, better not go to the river bank though they are depositing stones to harden it.",
          "They asked me to develop a machine learning algorithm that I was so afraid of doing. I ended up looking for a job.",
          "My old car is the best i.e. faithful, smooth, powerful and has never stalled on the highway.",
          "I had a dream that I was having a holiday in this island with the sea gulls flying by and sea breeze blowing.",
          "He is the strongest leader we have seen in this island nation and he gave us identity in the world."]

In [3]:
#text segment 1 chosen for experiment

text1 = sents1[0]
text2 = sents2[0]

In [4]:
text_list = [text1,text2]

In [5]:
# semantic vector of the corpus
def preprocess(text_list):
    stop_words = list(set(nltk.corpus.stopwords.words('english')))
    stop_words.extend(['(', ')', '[', ']', '{', '}', '.', ',', '!', '*','i.e'])
    stop_words.extend([chr(i) for i in range(97,123)])
    my_corpus = []
    ind_corpus = []
    tokenizer = RegexpTokenizer(r'\w+')
    for text in text_list:
        #tokenized_text = nltk.word_tokenize(text)
        text = text.lower()
        tokenized_text =tokenizer.tokenize(text)
        for word in stop_words:
            tokenized_text = list(filter(lambda a: a != word, tokenized_text))
        my_corpus.extend(tokenized_text)
        ind_corpus.append(tokenized_text)
        
    my_corpus = list(set(my_corpus))
    #print(my_corpus)
    return my_corpus,ind_corpus

# PART-1

In [6]:
#using spacy
# Similarity matrix using spaCy
def spacy_sim_matrix(vector):
    tokens = nlp(' '.join(vector))
    sim_mat = []
    for tok in tokens:
        sim_mat.append([tok.similarity(other_tok) for other_tok in tokens])
    sim_mat = pd.DataFrame(sim_mat, index=vector, columns=vector)                                 
    return sim_mat

In [7]:
my_corpus,ind_corpus= preprocess(text_list)
similarity_matrix = spacy_sim_matrix(my_corpus)
print(similarity_matrix)

               make     noise     lunch     front  classroom  everybody  \
make       1.000000  0.239830  0.224730  0.125662   0.291178   0.206717   
noise      0.239830  1.000000  0.541300  0.494886   0.514236   0.440464   
lunch      0.224730  0.541300  1.000000  0.576543   0.446029   0.350097   
front      0.125662  0.494886  0.576543  1.000000   0.275211   0.256349   
classroom  0.291178  0.514236  0.446029  0.275211   1.000000   0.512594   
everybody  0.206717  0.440464  0.350097  0.256349   0.512594   1.000000   
fry        0.414048  0.349236  0.402552  0.480363   0.342750   0.387895   
professor  0.341659  0.628171  0.585894  0.430072   0.697117   0.499030   
sea        0.218223  0.510816  0.463107  0.475711   0.469951   0.495240   
fish       0.199333  0.487422  0.399099  0.388916   0.578912   0.505522   
side       0.251184  0.387571  0.340186  0.403261   0.453875   0.510754   
ferocious  0.144574  0.265068  0.262756  0.299164   0.225927   0.221581   
like       0.195781  0.04

In [8]:
#PART-1
#using nltk

# semantic relatedness between words
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

    
def sem_relatedness_word(method,vector):
    entities = [wn.synsets(word)[0] for word in vector if wn.synsets(word)!=[] and wn.synsets(word)[0].pos() != 's']
    entity_names = [entity.name().split('.')[0] for entity in entities]
    similarities = []
    
    # Path Based Similarity
    if method == "path":
        similarities = [[entity.path_similarity(compared_entity) for compared_entity in entities] for entity in entities]
        
    elif method == 'lch':
        similarities = [[entity.lch_similarity(compared_entity) if entity.pos()==compared_entity.pos() else None for compared_entity in entities]
                        for entity in entities]
        
    elif method == 'wup':
        similarities = [[entity.wup_similarity(compared_entity) if  entity.pos()==compared_entity.pos() else None
                         for compared_entity in entities] for entity in entities]
    
    # Distribution Based Similarity
    elif method == 'res':
        similarities = [[entity.res_similarity(compared_entity, ic=brown_ic) if entity.pos()==compared_entity.pos() else None 
                         for compared_entity in entities] for entity in entities]
    
    elif method == 'jcn':
        similarities = [[entity.jcn_similarity(compared_entity, ic=brown_ic) if entity.pos()==compared_entity.pos() else None 
                         for compared_entity in entities] for entity in entities]
        
    elif method == 'lin':
        similarities = [[entity.lin_similarity(compared_entity, ic=brown_ic) if entity.pos()==compared_entity.pos() else None 
                         for compared_entity in entities] for entity in entities]
        
    similarity_frame = pd.DataFrame(similarities,
                                    index=entity_names, 
                                    columns=entity_names)
    similarity_frame.fillna(0,inplace=True)               
    print(similarity_frame)
        

In [9]:
#path_similarity
sem_relatedness_word('path',my_corpus)

              brand     noise     lunch     front  classroom       fry  \
brand      1.000000  0.083333  0.058824  0.052632   0.055556  0.058824   
noise      0.083333  1.000000  0.071429  0.062500   0.066667  0.071429   
lunch      0.058824  0.071429  1.000000  0.066667   0.071429  0.076923   
front      0.052632  0.062500  0.066667  1.000000   0.071429  0.066667   
classroom  0.055556  0.066667  0.071429  0.071429   1.000000  0.076923   
fry        0.058824  0.071429  0.076923  0.066667   0.076923  1.000000   
professor  0.055556  0.066667  0.071429  0.062500   0.071429  0.142857   
sea        0.071429  0.090909  0.100000  0.083333   0.090909  0.100000   
fish       0.050000  0.058824  0.062500  0.062500   0.076923  0.090909   
side       0.066667  0.083333  0.090909  0.142857   0.100000  0.090909   
like       0.333333  0.083333  0.058824  0.052632   0.055556  0.058824   

           professor       sea      fish      side      like  
brand       0.055556  0.071429  0.050000  0.0666

In [10]:
#lch_similarity
sem_relatedness_word('lch',my_corpus)

              brand     noise     lunch     front  classroom       fry  \
brand      3.637586  1.152680  0.804373  0.693147   0.747214  0.804373   
noise      1.152680  3.637586  0.998529  0.864997   0.929536  0.998529   
lunch      0.804373  0.998529  3.637586  0.929536   0.998529  1.072637   
front      0.693147  0.864997  0.929536  3.637586   0.998529  0.929536   
classroom  0.747214  0.929536  0.998529  0.998529   3.637586  1.072637   
fry        0.804373  0.998529  1.072637  0.929536   1.072637  3.637586   
professor  0.747214  0.929536  0.998529  0.864997   0.998529  1.691676   
sea        0.998529  1.239691  1.335001  1.152680   1.239691  1.335001   
fish       0.641854  0.804373  0.864997  0.864997   1.072637  1.239691   
side       0.929536  1.152680  1.239691  1.691676   1.335001  1.239691   
like       2.538974  1.152680  0.804373  0.693147   0.747214  0.804373   

           professor       sea      fish      side      like  
brand       0.747214  0.998529  0.641854  0.9295

In [11]:
#wup_similarity
sem_relatedness_word('wup',my_corpus)

              brand     noise     lunch     front  classroom       fry  \
brand      1.000000  0.352941  0.111111  0.100000   0.105263  0.111111   
noise      0.352941  1.000000  0.133333  0.117647   0.125000  0.133333   
lunch      0.111111  0.133333  1.000000  0.222222   0.235294  0.250000   
front      0.100000  0.117647  0.222222  1.000000   0.315789  0.300000   
classroom  0.105263  0.125000  0.235294  0.315789   1.000000  0.400000   
fry        0.111111  0.133333  0.250000  0.300000   0.400000  1.000000   
professor  0.105263  0.125000  0.235294  0.285714   0.380952  0.521739   
sea        0.133333  0.166667  0.307692  0.266667   0.285714  0.307692   
fish       0.095238  0.111111  0.210526  0.285714   0.400000  0.545455   
side       0.125000  0.153846  0.285714  0.625000   0.400000  0.375000   
like       0.900000  0.352941  0.111111  0.100000   0.105263  0.111111   

           professor       sea      fish      side      like  
brand       0.105263  0.133333  0.095238  0.1250

In [12]:
#res_similarity
sem_relatedness_word('res',my_corpus)

              brand     noise      lunch     front  classroom        fry  \
brand      7.732198  1.592755  -0.000000 -0.000000  -0.000000  -0.000000   
noise      1.592755  7.741767  -0.000000 -0.000000  -0.000000  -0.000000   
lunch     -0.000000 -0.000000  10.371255  0.801759   0.801759   0.801759   
front     -0.000000 -0.000000   0.801759  8.137663   1.290026   1.290026   
classroom -0.000000 -0.000000   0.801759  1.290026  11.169763   1.531834   
fry       -0.000000 -0.000000   0.801759  1.290026   1.531834  13.772453   
professor -0.000000 -0.000000   0.801759  1.290026   1.531834   6.394069   
sea       -0.000000 -0.000000   0.801759  0.801759   0.801759   0.801759   
fish      -0.000000 -0.000000   0.801759  1.290026   1.531834   2.224150   
side      -0.000000 -0.000000   0.801759  4.944545   1.290026   1.290026   
like       6.421295  1.592755  -0.000000 -0.000000  -0.000000  -0.000000   

           professor       sea      fish      side       like  
brand      -0.000000 -0

In [13]:
#jcn_similarity
sem_relatedness_word('jcn',my_corpus)

                   brand          noise          lunch          front  \
brand      1.000000e+300   8.137719e-02   5.523808e-02   6.301252e-02   
noise       8.137719e-02  1.000000e+300   5.520890e-02   6.297455e-02   
lunch       5.523808e-02   5.520890e-02  1.000000e+300   5.915270e-02   
front       6.301252e-02   6.297455e-02   5.915270e-02  1.000000e+300   
classroom   5.290456e-02   5.287779e-02   5.015674e-02   5.978224e-02   
fry         4.650157e-02   4.648089e-02   4.436520e-02   5.173288e-02   
professor   5.600988e-02   5.597988e-02   5.293938e-02   6.377793e-02   
sea         5.802163e-02   5.798943e-02   5.473306e-02   6.235619e-02   
fish        6.309682e-02   6.305875e-02   5.922698e-02   7.313110e-02   
side        6.539348e-02   6.535258e-02   6.124605e-02   1.721639e-01   
like        1.697945e-01   6.428337e-02   4.679212e-02   5.225336e-02   

               classroom            fry      professor            sea  \
brand       5.290456e-02   4.650157e-02   5.600988

In [14]:
#lin_similarity
sem_relatedness_word('lin',my_corpus)

              brand     noise     lunch     front  classroom       fry  \
brand      1.000000  0.205863 -0.000000 -0.000000  -0.000000 -0.000000   
noise      0.205863  1.000000 -0.000000 -0.000000  -0.000000 -0.000000   
lunch     -0.000000 -0.000000  1.000000  0.086635   0.074440  0.066416   
front     -0.000000 -0.000000  0.086635  1.000000   0.133630  0.117756   
classroom -0.000000 -0.000000  0.074440  0.133630   1.000000  0.122831   
fry       -0.000000 -0.000000  0.066416  0.117756   0.122831  1.000000   
professor -0.000000 -0.000000  0.078247  0.141299   0.143891  0.535197   
sea       -0.000000 -0.000000  0.080684  0.090900   0.077568  0.068894   
fish      -0.000000 -0.000000  0.086734  0.158732   0.158853  0.203222   
side      -0.000000 -0.000000  0.089427  0.629978   0.137753  0.120946   
like       0.685594  0.169970 -0.000000 -0.000000  -0.000000 -0.000000   

           professor       sea      fish      side      like  
brand      -0.000000 -0.000000 -0.000000 -0.0000

# PART - 2

In [15]:
# semantic vector representing text segments
def sem_vector(my_corpus, ind_corpus):
    sim_mat = spacy_sim_matrix(my_corpus)
    threshold = max(sim_mat.mean())
    v = [[], []]
    
    for i in range(2):
        for word in my_corpus:
            if word in ind_corpus[i]:
                v[i].append(1)
            else:
                max_sim = 0
                for j in ind_corpus[i]:
                    sim = nlp(word).similarity(nlp(j))
                    if sim > max_sim:
                        max_sim = sim
                        sim_word = j
                if max_sim > threshold:
                    v[i].append(max_sim)
                else:
                    v[i].append(0)
    return v

# PART-3

In [16]:
#semantic relatedness between text segements

for i, sent1, sent2 in zip([i+1 for i in range(len(sents1))], sents1, sents2):
    p = preprocess([sent1, sent2])[1]
    v1,v2 = sem_vector(list(set(p[0] + p[1])),[p[0], p[1]])
    print("Semantic relatedness between sentences "+str(i)+" :", cosine_similarity([v1],[v2])[0][0], '\n')

Semantic relatedness between sentences 1 : 0.8464173143763345 

Semantic relatedness between sentences 2 : 0.8475331068550017 

Semantic relatedness between sentences 3 : 0.9382694389292592 

Semantic relatedness between sentences 4 : 0.8332144818258096 

Semantic relatedness between sentences 5 : 0.9192209122283185 

Semantic relatedness between sentences 6 : 0.8961191566636407 



# PART-4

In [17]:
# word sense disambiguation
from nltk.corpus import wordnet as wn, wordnet_ic
from nltk.wsd import lesk
from nltk import word_tokenize
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

def after_wsd_similarity(sample1, sample2):
    #sentences = [sample1, sample2]

    similar_words = [i for i in word_tokenize(sample1.lower()) for j in word_tokenize(sample2.lower()) if i==j]
    similar_words = list(set([i for i in similar_words if i not in ['.', ',']]))

    print('SENTENCES:', '\n', sample1,'\n', sample2)
    print("COMMON_LEMMA :", similar_words)
    similarity, n = 0, 0
    senses = []    
    
    for word in similar_words:
        senses = []
        for sentence in [sample1, sample2]:
            word_syn = lesk(word_tokenize(sentence.lower()), word)
            #print ('Word synset:', word_syn)
            senses.append((word_syn, word_syn.name().split('.')[1]))

        if senses[0][1] == senses[1][1]:
            similarity += senses[0][0].path_similarity(senses[1][0],semcor_ic)
            n += 1
    if n != 0:
        return (similarity/n)
    else:
        return 0

In [18]:
for sent1, sent2 in zip(sents1, sents2):
    p = preprocess([sent1, sent2])[1]
    after = after_wsd_similarity(" ".join(p[0]), " ".join(p[1]))
    print('SIM. AFTER WSD :', after)
    
    v1,v2 = sem_vector(list(set(p[0] + p[1])),[p[0], p[1]])
    before = cosine_similarity([v1],[v2])[0][0]
    print("SIM. BEFORE WSD :", before)
    
    ensemble = after * 0.3 + before * 0.7
    print("SIM. WITH ENSEMBLE :", ensemble, '\n')

SENTENCES: 
 like fish fry lunch whenever side sea 
 professor ferocious make noise classroom fry front everybody
COMMON_LEMMA : ['fry']
SIM. AFTER WSD : 1.0
SIM. BEFORE WSD : 0.8464173143763345
SIM. WITH ENSEMBLE : 0.8924921200634341 

SENTENCES: 
 visiting bank deposit cash earned day earn interest overtime 
 flood better go river bank though depositing stones harden
COMMON_LEMMA : ['bank']
SIM. AFTER WSD : 1.0
SIM. BEFORE WSD : 0.8475331068550017
SIM. WITH ENSEMBLE : 0.8932731747985012 

SENTENCES: 
 pure malt whiskey bit soda make afraid make brave 
 asked develop machine learning algorithm afraid ended looking job
COMMON_LEMMA : ['afraid']
SIM. AFTER WSD : 1.0
SIM. BEFORE WSD : 0.9382694389292592
SIM. WITH ENSEMBLE : 0.9567886072504814 

SENTENCES: 
 horse faithful powerful car loves cares talks makes feel good upset 
 old car best faithful smooth powerful never stalled highway
COMMON_LEMMA : ['car', 'faithful', 'powerful']
SIM. AFTER WSD : 1.0
SIM. BEFORE WSD : 0.8332144818258096

# step - 4 observations

Comparing the semantic relatedness before and after wsd, 
we get to the fact that the common lemma occuring in both the sentence segments with same POS tags
always have the same senses. This can be observed from the values achieved above.

Semantic relatedness after wsd turned out to be 100% equal, meaning that the common lemma with same POS tags
are not the differentiating words between the sentences above, instead all the words in the sentence matter
when we are finding the semantic relatedness

So, when finding the ensemble values for semantic relatedness, more weightage was given the method without wsd,
causing it to have more effect on the result than the method with wsd

This can vary from dataset to dataset


# STEP-5
## Decisions

Coming to Path Based Similarity:
    
    WUP similarity seems to be better as compared to path_similarity and lch_similarity as WUP calculates the 
    relatedness by looking at the depth of the two synsets in the WordNet. The score can never be 0 as the depth of the 
    LCS (Least Common Subsumer) is never 0.
    
Coming to Distribution Based Similarity:
    
    res_similarity is not good as it is not scaled between 0-1 and for the same words,
    Ex: sim(professor-professor) = 10.212 and sim(lunch-lunch)= 10.371255
    
    lin_similarity seems to be better because it captures not only the commonality but also differences.